In [1]:
import pymysql.cursors # for load data from MySQL

import time

import pandas as pd
import numpy as np
import re
import csv
import geocoder

## Load data from MySQL

In [31]:
# Connect to the database
connection = pymysql.connect(host='',
                             user='',
                             password='',
                             db='',
                             charset='',
                             cursorclass=pymysql.cursors.DictCursor)

# try:
#     with connection.cursor() as cursor:
#         # Create a new record
#         query = "SHOW TABLES"
#         cursor.execute(query)
#         result = cursor.fetchone()
#         print(result)

#     # connection is not autocommit by default. So you must commit to save
#     # your changes.
#     connection.commit()

#     with connection.cursor() as cursor:
#         # Read a single record
#         query = "SELECT * FROM "
#         cursor.execute(query)
#         result = cursor.fetchone()
#         print(result)
# finally:
#     connection.close()

In [32]:
with connection.cursor() as cursor:
    query = "SHOW TABLES"
    cursor.execute(query)
    result = cursor.fetchall()
    print(result)

[{'Tables_in_greenpeacedatahub': 'air_pollution_2013'}, {'Tables_in_greenpeacedatahub': 'air_pollution_2014'}, {'Tables_in_greenpeacedatahub': 'air_pollution_2015'}, {'Tables_in_greenpeacedatahub': 'bj_dangerous_chemicals_production_authorized_enterprise'}, {'Tables_in_greenpeacedatahub': 'dangerous_chemicals_production_license'}, {'Tables_in_greenpeacedatahub': 'detox_hebei_2014'}, {'Tables_in_greenpeacedatahub': 'detox_hebei_2016'}, {'Tables_in_greenpeacedatahub': 'detox_hebei_enterprise_2015'}, {'Tables_in_greenpeacedatahub': 'detox_hebei_enterprise_2016'}, {'Tables_in_greenpeacedatahub': 'detox_hebei_enterprise_2016_plus'}, {'Tables_in_greenpeacedatahub': 'detox_hebei_enterprise_2016_seasonly_production'}, {'Tables_in_greenpeacedatahub': 'detox_hebei_enterprise_2017'}, {'Tables_in_greenpeacedatahub': 'detox_hebei_enterprise_2017_fqsite'}, {'Tables_in_greenpeacedatahub': 'detox_hebei_enterprise_2017_fssite'}, {'Tables_in_greenpeacedatahub': 'detox_hebei_enterprise_2017_plus'}, {'Tab

In [11]:
def table_to_pandas(conn, table_name):
    with conn.cursor() as cursor:
        query = "SELECT * FROM {}".format(table_name)
        cursor.execute(query)
        result = cursor.fetchall()
    return pd.DataFrame(result)

In [ ]:
air_pollution = dict()

for table in ['air_pollution_2013', 'air_pollution_2014', 'air_pollution_2015']:
    print(table)
    air_pollution[table] = table_to_pandas(conn=connection, table_name=table)

## Load data from local

In [2]:
ap_2013 = pd.read_csv('./tables/air_pollution_2013.csv')

In [5]:
ap_2014 = pd.read_csv('./tables/PM25_20160101_2014only.csv')

In [ ]:
ap_2015 = pd.read_csv('./tables/PM25_20160101_2014only.csv')

## (1) Dataset = ap_2013 

### Data Mining and Cleaning

In [3]:
ap_2013.shape

(26735, 5)

In [4]:
ap_2013.head(5)

,Unnamed: 0,city,id,pm25,update_time
0,0,﻿中山,1,NaN,2013-01-01 00:00:00\n
1,1,中山,2,NaN,2013-01-02 00:00:00\n
2,2,中山,3,NaN,2013-01-03 00:00:00\n
3,3,中山,4,NaN,2013-01-04 00:00:00\n
4,4,中山,5,NaN,2013-01-05 00:00:00\n


In [5]:
len(list(set(ap_2013['city'])))

75

* update_time

In [6]:
ap_2013['update_time'] = pd.to_datetime(ap_2013['update_time'].apply(lambda x: x.strip()))

In [59]:
# import datetime

# to_datetime = lambda x: datetime.datetime.strptime(x.strip(), 
#                                                    '%Y-%m-%d  %H:%M:%S')

# ap_2013_tmp['update_time'] = pd.to_datetime(ap_2013_tmp['update_time'].apply(to_datetime))

* city

strip whitespaces from **city**

In [7]:
ap_2013['city'] = ap_2013.city.map(lambda x: re.findall(r'[\u4e00-\u9fff]+', x)[0])

create new column lat and lng of city

In [14]:
# city_latlng = list()

# for x in list(set(ap_2013['city'])):
#     row = dict()
#     row['city'] = x
#     row['lat'], row['lng'] = geocoder.google(x).latlng
#     city_latlng.append(row)
#     time.sleep(0)

In [16]:
# with open('./data/city_latlng.csv', 'w') as f:  
#     w = csv.DictWriter(f, city_latlng[0].keys())
#     w.writeheader()
#     for row in city_latlng:
#         w.writerow(row)

In [17]:
# df_city_latlng = pd.DataFrame(city_latlng)

In [18]:
df_city_latlng = pd.read_csv('./data/city_latlng.csv')

In [19]:
ap_2013_new = pd.merge(ap_2013, df_city_latlng, how='left', on=['city'])

### EDA

In [111]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [112]:
from bokeh.layouts import row, widgetbox
from bokeh.models import Select
#from bokeh.charts import Histogram
from bokeh.io import show
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

In [191]:
import plotly.plotly as py
from plotly.graph_objs import Scatter, Figure, Layout
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [192]:
import seaborn as sns

/usr/local/lib/python3.6/site-packages/matplotlib/font_manager.py:281: UserWarning:

Matplotlib is building the font cache using fc-list. This may take a moment.



* Sample: Beijing daily PM 2.5 levels and analysis

In [106]:
ap_2013[ap_2013_new['city'] == '北京']['pm25'].describe()

count    355.000000
mean      90.101408
std       76.131546
min        6.000000
25%       37.000000
50%       68.000000
75%      124.000000
max      646.000000
Name: pm25, dtype: float64

In [113]:
p1 = figure(x_axis_type="datetime", title="Beijing Daily PM 2.5 Levels 2013")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'PM 2.5'

p1.line(ap_2013[ap_2013_new['city'] == '北京']['update_time'], ap_2013[ap_2013_new['city'] == '北京']['pm25'])

GlyphRenderer(id='ccd61e2e-3e35-4865-8e14-755d93906147', ...)

In [114]:
show(p1)

* Sample: Beijing monthly PM 2.5 levels 2013 and statistical and analysis

In [138]:
data_bp = ap_2013.groupby(ap_2013[ap_2013_new['city'] == '北京']['update_time'].dt.month)\
['pm25'].describe()

data_bp['month'] = data_bp.index

data_bp

,count,mean,std,min,25%,50%,75%,max,month
update_time,,,,,,,,,
1.0,21.0,198.809524,143.162362,18.0,104.00,174.0,266.00,646.0,1.0
2.0,28.0,99.392857,81.939607,10.0,31.75,77.5,140.25,280.0,2.0
3.0,31.0,107.096774,80.669017,6.0,41.00,80.0,168.50,289.0,3.0
4.0,30.0,56.633333,45.395946,7.0,20.00,46.0,77.50,188.0,4.0
5.0,31.0,79.935484,51.323117,13.0,47.00,68.0,105.00,258.0,5.0
6.0,30.0,102.766667,67.574574,8.0,47.25,102.5,131.00,301.0,6.0
7.0,31.0,67.774194,41.324496,13.0,35.00,57.0,99.50,161.0,7.0
8.0,31.0,62.548387,43.059524,6.0,29.50,47.0,86.50,167.0,8.0
9.0,30.0,87.433333,63.731486,16.0,38.50,72.0,101.75,252.0,9.0


In [149]:
pm_bar = [go.Bar(
    x = data_bp['month'],
    y = data_bp['mean'],
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6
)]

layout = go.Layout(
    title= 'Beijing PM 2.5, Monthly Average (2013)'
)

fig = go.Figure(data=pm_bar, layout = layout)

iplot(fig, filename = 'basic-bar')

In [207]:
month = ['January', 'February', 'March', 'April', 'May', 
         'June', 'July', 'August', 'September', 'October', 
         'November', 'December']

traces = []

for x in range(1,12):
    traces.append(go.Box(
        y = ap_2013_new[(ap_2013_new['city'] == '北京') &\
            (ap_2013_new['update_time'].dt.month == x)]['pm25']
    ))
    
    
iplot(traces, filename='pandas-box-plot')